In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from cartiflette import carti_download
import plotly.express as px

In [ ]:
df_patientele = pd.read_csv("Bases de données/patientele.csv", sep=";")
df_patientele['nombre_patients_uniques'] = pd.to_numeric(df_patientele['nombre_patients_uniques'],errors='coerce')
df_patientele.columns = df_patientele.columns.str.replace("\ufeff", "", regex=False).str.strip()

df_patientele_2017 = df_patientele[(df_patientele['annee'] == 2017) & (df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]           

In [ ]:
df_patientele.head()

## Carte des pays différents

In [ ]:
# Téléchargement des informations géospatiales des départements français (DROM compris)

departements = carti_download(
    values = ["France"],
    crs = 4326,
    borders = "DEPARTEMENT",
    vectorfile_format="geojson",
    simplification=50,
    filter_by="FRANCE_ENTIERE_DROM_RAPPROCHES",
    source="EXPRESS-COG-CARTO-TERRITOIRE",
    year=2022
)

# On modifie le code des départements : on rajoute un 0 à ceux qui ont un code à deux chiffres
#departements['INSEE_DEP'] = departements['INSEE_DEP'].str.zfill(3)

# Test d'affichage de la carte
departements.plot().axis('off')

In [ ]:
Carte_2017 = df_patientele_2017.copy()
Carte_2017['INSEE_DEP'] = Carte_2017['departement'].str[:3].astype(str)
Carte_2017 = departements.merge(Carte_2017, on='INSEE_DEP', how='left')

In [ ]:
# Tracé pour l'année 2017

vmin = Carte_2017['nombre_patients_uniques'].quantile(0.05)
vmax = Carte_2017['nombre_patients_uniques'].quantile(0.95)

fig, ax = plt.subplots(1, 1, figsize=(10, 10))
Carte_2017.plot(
    column='nombre_patients_uniques', 
    cmap='OrRd',
    vmin=vmin,
    vmax=vmax,      
    linewidth=0.4,
    edgecolor='black',
    legend=True,       
    ax=ax
)

# On supprime les axes
ax.axis('off')  
ax.set_title("Densité par département", fontsize=15)
plt.show()

## Graphique Evolution par année

In [ ]:
df_medecin = df_patientele[(df_patientele['profession_sante'] == "Médecins généralistes (hors médecins à expertise particulière - MEP)")]  
df_graphique_évol = df_medecin[["annee","libelle_departement","patients_uniques_integer"]]
df_graphique_évol.info

In [ ]:
L = []

df = df_graphique_évol[df_graphique_évol["libelle_departement"] == "Paris"]
df.sort_values(by='annee')
df.head()

plt.plot(df['annee'],
        df['patients_uniques_integer'],
        label=region,
        alpha=0.7
    )

plt.xlabel("Année")
plt.ylabel("Nombre de patients uniques")
plt.title("Évolution du nombre de patients uniques par région")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
df.head(10)
df.shape
df['annee'].value_counts().sort_index()

In [ ]:
df_graphique_évol = df_graphique_évol.sort_values(by='annee')

for region in df_graphique_évol['libelle_departement'].unique():
    tmp = df_graphique_évol[df_graphique_évol['libelle_departement'] == region]
    plt.plot(
        tmp['annee'],
        tmp['patients_uniques_integer'],
        label=region,
        alpha=0.7
    )

plt.xlabel("Année")
plt.ylabel("Nombre de patients uniques")
plt.title("Évolution du nombre de patients uniques par région")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
df.head(10)
df.shape
df['annee'].value_counts().sort_index()